In [1]:
from bs4 import BeautifulSoup as soup
from requests import get
import pandas as pd
import requests
import re

In [2]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
df5 = pd.DataFrame()
df6 = pd.DataFrame()
df7 = pd.DataFrame()
df8 = pd.DataFrame()
df9 = pd.DataFrame()
df10 = pd.DataFrame()
df11 = pd.DataFrame()

In [26]:
title = []
locationnew=[]
price=[]
price_per_sqft=[]
area_in_sqft=[]
building_status=[]
n_pages = 0
for page in range(1,100):  #The no. of pages you wanted to scrap considering pages available in the url.
    n_pages += 1
    #url = 'https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?budget=,'+'&page='+str(page)
    if((page == 1) | (page==2) ):
        url='https://www.makaan.com/bangalore-property/electronic-city-phase-2-flats-for-sale-68601'
    else:
        url='https://www.makaan.com/bangalore-property/electronic-city-phase-2-flats-for-sale-68601?page='+'str(page)'
    # This for gets us into next page after every iteration.
    res = requests.get(url) # Here r is response data.
    
    page_html = soup(res.text, 'html.parser') 
    house_containers = page_html.find_all("li",class_="cardholder") # Class containing overall data of a property.
    for data in house_containers:
        type=data.find_all("a",class_="typelink") # Class containing title name of a property.
        for i in type:
            d=i.text
            title.append(d) 
        location=data.find_all("a",class_="loclink")
        #print(location)
        for i in location:
            
            loc = i.find("span",itemprop="addressLocality").text
            locationnew.append(loc)

        cost=data.find_all("td",class_="price")
        for i in cost:
            d=i.text
            if 'L' in d:
                a = d.split()[0]
                price.append(float(a)) # Type conversion of obtained data.
            elif 'Cr' in d:
                b = d.split()[0]
                price.append(float(b) * 100) # Coverting CRORES into LAKHS.
            else:  # If no value found in the class return value 0.
                e=0
                price.append(float(e)) 
                
        rate_sqft=data.find_all('td',class_="lbl rate")
        for i in rate_sqft:
            d=i.text
            res= d.split('/',)[0]
            r=re.sub(",","" ,res) # Substitute no character in the place of ,(comma).
            price_per_sqft.append(r)
            
        area_sqft=data.find_all('td',class_="size")
        for i in area_sqft:
            d=i.text
            area_in_sqft.append(d)
            
        building_status_=data.find_all("td",class_="val")
        for i in building_status_:
            d=i.text
            building_status.append(d)
df = pd.DataFrame()            
df['title']=title 
df['location']=locationnew
df['price(L)']=price
df['rate_persqft']=price_per_sqft
df['area_insqft']=area_in_sqft
df['building_status']=building_status

#print('You scraped {} pages containing {} properties.'.format(n_pages, len(title)))
# Returns the number of pages scrapped and time taken for scarpping those pages.
df = df.loc[~df.duplicated()]

In [27]:
df.to_csv('df.csv',index= False)
